# What is Flask?

A lightweight Python web framework [Flask][flask]

- Very little boilerplate
- Many choices for storage back-ends, but none included in Flask itself

[flask]: http://flask.pocoo.org/


# ReST: Representational State Transfer

Everyone *says* they're restful, but...

REST has a few attributes that make it truly RESTful:

- URLs represent *resources* (i.e. nouns), not *actions* (i.e. verbs)
    - a POST to `/mailingList/createUser` is **not** RESTful
- Resources may have one or more *representations* (formats transferred over the web)
    - HTML
    - XML
    - JSON
- HTTP verbs are used to represent operations on resources
    - GET - safe & idempotent, give me the representation for a resource
    - PUT - idempotent, replace a resource at a given URL
    - PATCH - partial update to a resource at a given URL
    - DELETE - idempotent, delete a resource at a given URL
    - POST - do something else (frequently create)
- Representations must communicate their relationships to other resources via *hypertext*
    - Generally, this means representations have URLs to relate to other resources, *not* just IDs
    - Requiring knowledge of URL layout is not RESTful
    - "Hypertext as the engine of application state" or HATEOAS
    - We'll use a tiny part of a HATEOAS standard known as HAL

# Getting started - our first Flask API

- Single endpoint/resource: the root (/)
- Single representation: `{"hello": "world"}`
- Single operation: GET
- A flask **view** is a function that maps to a URL

In [1]:
!pip install flask

Looking in links: /Users/rick446/src/wheelhouse
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Normally, we would run flask by executing the following in the shell:


```bash
$ FLASK_APP=intro_flask.app1 FLASK_ENV=development flask run
```

Since we're using the notebook, here's a little program to run a subprocess in a thread:

In [2]:
!which flask

/Users/rick446/.virtualenvs/py37/bin/flask


In [3]:
import os, sys, threading, subprocess

def output_thread(proc):
    for line in proc.stdout:
        print(line.decode('utf-8'), end='')
    print('Exiting output thread')

def run_flask_app(app_name):
    proc = subprocess.Popen(
        # [sys.executable, 'flask', 'run'],
        ['flask', 'run', '--no-reload'],
        env={
            **os.environ, 
            'FLASK_APP': app_name,
            'FLASK_ENV': 'development',
        },
        stderr=subprocess.STDOUT,
        stdout=subprocess.PIPE
    )
    # Wait for the port to bind
    for line in proc.stdout:
        line = line.decode('utf-8')
        print(line, end='')
        if ' * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)' in line:
            break
    else:
        print('== Error starting server ==')
        return None
    thd = threading.Thread(target=output_thread, args=(proc,))
    thd.setDaemon(True)
    thd.start()
    return proc


In [4]:
%%file data/flask-examples/app1.py
import flask

app = flask.Flask(__name__)

@app.route('/')
def get_root():
    return flask.jsonify(hello='world')

Overwriting data/flask-examples/app1.py


In [5]:
sp = run_flask_app('data.flask-examples.app1')

 * Serving Flask app "data.flask-examples.app1"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


In [6]:
import requests
resp = requests.get('http://localhost:5000')
resp.json()

127.0.0.1 - - [29/Jul/2020 09:50:26] "GET / HTTP/1.1" 200 -


{'hello': 'world'}

In [7]:
!curl localhost:5000

127.0.0.1 - - [29/Jul/2020 09:50:34] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}


In [8]:
sp.kill()

Exiting output thread


WSGI - web server gateway interface

# One more helper: run an app in a context manager

In [9]:
import time, contextlib

@contextlib.contextmanager
def running_app(app_name):
    proc = run_flask_app(app_name)
    try:
        yield proc
    finally:
        proc.kill()        

In [10]:
with running_app('data.flask-examples.app1'):
    !curl localhost:5000

 * Serving Flask app "data.flask-examples.app1"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 09:53:24] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
Exiting output thread


# Handling url parameters

In [15]:
%%file data/flask-examples/app2.py
import flask

app = flask.Flask(__name__)

@app.route('/')
def get_root():
    return flask.jsonify(hello='world')

@app.route('/<name>')
def get_name(name):
    return flask.jsonify(hello=name)

Overwriting data/flask-examples/app2.py


In [16]:
with running_app('data.flask-examples.app2'):
    !curl localhost:5000
    !curl localhost:5000/rick
    !curl localhost:5000/intuit

 * Serving Flask app "data.flask-examples.app2"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 09:55:07] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
127.0.0.1 - - [29/Jul/2020 09:55:07] "GET /rick HTTP/1.1" 200 -
{
  "hello": "rick"
}
127.0.0.1 - - [29/Jul/2020 09:55:07] "GET /intuit HTTP/1.1" 200 -
{
  "hello": "intuit"
}
Exiting output thread


# Handling JSON data

The code used below requires that the request be sent with `Content-Type: application/json`

In [17]:
%%file data/flask-examples/app3.py
import flask

app = flask.Flask(__name__)

state = {'name': 'world'}

@app.route('/')
def get_root():
    return flask.jsonify(hello=state['name'])

@app.route('/', methods=['PUT'])
def set_name():
    body = flask.request.json  # resolves to None if no valid JSON Content-Type header
    state['name'] = body['name']
    return flask.jsonify(name=state['name'])

Overwriting data/flask-examples/app3.py


In [22]:
with running_app('data.flask-examples.app3'):
    !curl localhost:5000
    !curl -XPUT -H 'Content-Type: application/json' -d '{"name": "Rick"}' localhost:5000 
    !curl localhost:5000
    requests.put('http://localhost:5000', json=dict(name='intuit'))
    !curl localhost:5000

 * Serving Flask app "data.flask-examples.app3"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 09:59:11] "GET / HTTP/1.1" 200 -
{
  "hello": "world"
}
127.0.0.1 - - [29/Jul/2020 09:59:12] "PUT / HTTP/1.1" 200 -
{
  "name": "Rick"
}
127.0.0.1 - - [29/Jul/2020 09:59:12] "GET / HTTP/1.1" 200 -
{
  "hello": "Rick"
}
127.0.0.1 - - [29/Jul/2020 09:59:12] "PUT / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 09:59:12] "GET / HTTP/1.1" 200 -
{
  "hello": "intuit"
}
Exiting output thread


# Using auth data (basic) & URL generation

In [ ]:
%%file data/flask-examples/app4.py
import flask

app = flask.Flask(__name__)

@app.route('/profile/<username>')
def get_profile(username):
    return flask.jsonify(username=username)

@app.route('/userinfo')
def get_userinfo():
    username = flask.request.authorization['username']
    profile_url = flask.url_for(
        'get_profile', username=username, 
        _external=True
    )
    return flask.jsonify(
        _links={'profile': profile_url},
        username=username,
        # don't do this in production, obviously
        password=flask.request.authorization['password'] ,
        headers=dict(flask.request.headers)
    )

In [24]:
with running_app('data.flask-examples.app4'):
    !curl rick:secret@localhost:5000/userinfo
    print(requests.get('http://localhost:5000/userinfo', auth=("intuit", "rocks")).json())

 * Serving Flask app "data.flask-examples.app4"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:01:37] "GET /userinfo HTTP/1.1" 200 -
{
  "_links": {
    "profile": "http://localhost:5000/profile/rick"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Authorization": "Basic cmljazpzZWNyZXQ=", 
    "Host": "localhost:5000", 
    "User-Agent": "curl/7.54.0"
  }, 
  "password": "secret", 
  "username": "rick"
}
127.0.0.1 - - [29/Jul/2020 10:01:37] "GET /userinfo HTTP/1.1" 200 -
{'_links': {'profile': 'http://localhost:5000/profile/intuit'}, 'headers': {'Accept': '*/*', 'Accept-Encoding': 'gzip, deflate', 'Authorization': 'Basic aW50dWl0OnJvY2tz', 'Connection': 'keep-alive', 'Host': 'localhost:5000', 'User-Agent': 'python-requests/2.24.0'}, 'password': 'rocks', 'username': 'intuit'}
Exiting output thread


# The task: build a REST API for a blog in Flask

- What are our resources?
- What are the operations on the resources?
- What representation(s) do we want to use for each resource?

# Building the blog post API

## Resource structure

```
{
    _links: {self: <link_to_post>},
    postedDate: ...,
    authorName: ...,
    title: ...,
    body: ...,
}
```

## URL structure

- / 
    - GET: return `{_links: {posts: /post}}`
- /post
    - GET: return list of posts
    - POST: create and return a post
- `/post/<post_id>` - return a single post
    - GET: return the post
    - PUT: update the post
    - DELETE: delete the post



In [25]:
%%file data/flask-examples/app5.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Flask, jsonify, request, abort


app = Flask(__name__)

state = {
    'posts': {
        uuid4().hex: {
            'postedDate': datetime.utcnow(),
            'authorName': 'rick',
            'title': 'first!',
            'body': 'First post!'
        }
    }
}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

# /post/123 - *not* _external=True
# http://localhost:5000/post/123 - _external=True


@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('get_posts')})

@app.route('/post')
def get_posts():
    post_links = [
        url_for('get_post', id=id) 
        for id in state['posts']
    ]
    return jsonify(
        _links={'self': url_for('get_posts')},
        data=[
            dict(_links=dict(self=link)) 
            for link in post_links
        ]
    )

@app.route('/post/<id>')
def get_post(id):
    # post = state['posts'][id]  # could generate a KeyError => 500 Error
    post = state['posts'].get(id)
    if not post:
        abort(404)
    return jsonify(
        _links={'self': url_for('get_post', id=id)},
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body']
    )

Overwriting data/flask-examples/app5.py


In [26]:
with running_app('data.flask-examples.app5'):
    !curl localhost:5000
    !curl localhost:5000/post
    print('Got post, now need to return...')

 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:13:10] "GET / HTTP/1.1" 200 -
{
  "_links": {
    "posts": "http://localhost:5000/post"
  }
}
127.0.0.1 - - [29/Jul/2020 10:13:10] "GET /post HTTP/1.1" 200 -
{
  "_links": {
    "self": "http://localhost:5000/post"
  }, 
  "data": [
    {
      "_links": {
        "self": "http://localhost:5000/post/78dba9ffcfa243d4949c7c9e67fe550a"
      }
    }
  ]
}
Got post, now need to return...
Exiting output thread


In [27]:
import requests

with running_app('data.flask-examples.app5'):
    root = requests.get('http://localhost:5000').json()
    print('root is', root)
    posts = requests.get(root['_links']['posts']).json()
    print('posts is', posts)
    post = requests.get(posts['data'][0]['_links']['self']).json()
    print('post is', post)


 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:14:25] "GET / HTTP/1.1" 200 -
root is {'_links': {'posts': 'http://localhost:5000/post'}}
127.0.0.1 - - [29/Jul/2020 10:14:25] "GET /post HTTP/1.1" 200 -
posts is {'_links': {'self': 'http://localhost:5000/post'}, 'data': [{'_links': {'self': 'http://localhost:5000/post/8e1cd80a07904ecfad57b8438ecd687a'}}]}
127.0.0.1 - - [29/Jul/2020 10:14:25] "GET /post/8e1cd80a07904ecfad57b8438ecd687a HTTP/1.1" 200 -
post is {'_links': {'self': 'http://localhost:5000/post/8e1cd80a07904ecfad57b8438ecd687a'}, 'authorName': 'rick', 'body': 'First post!', 'postedDate': '2020-07-29T17:14:25.373751', 'title': 'first!'}
Exiting output thread


In [28]:
post

{'_links': {'self': 'http://localhost:5000/post/8e1cd80a07904ecfad57b8438ecd687a'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2020-07-29T17:14:25.373751',
 'title': 'first!'}

# Aside: `glom` can be handy for consuming deeply nested dicts like this

In [29]:
!pip install glom

Looking in links: /Users/rick446/src/wheelhouse
You are using pip version 19.0.3, however version 20.2b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
from glom import glom

In [31]:
import requests

with running_app('data.flask-examples.app5'):
    root = requests.get('http://localhost:5000').json()
    posts = requests.get(glom(root, '_links.posts')).json()
    post = requests.get(glom(posts, 'data.0._links.self')).json()
post

 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:16:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:16:20] "GET /post HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:16:20] "GET /post/3b4384c932ef4f0c9868053c6c9bc68c HTTP/1.1" 200 -


{'_links': {'self': 'http://localhost:5000/post/3b4384c932ef4f0c9868053c6c9bc68c'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2020-07-29T17:16:20.544407',
 'title': 'first!'}

Exiting output thread


In [32]:
with running_app('data.flask-examples.app5'):
    !curl -X POST localhost:5000 -d ''

 * Serving Flask app "data.flask-examples.app5"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:16:46] "POST / HTTP/1.1" 405 -
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>405 Method Not Allowed</title>
<h1>Method Not Allowed</h1>
<p>The method is not allowed for the requested URL.</p>
Exiting output thread


## Modifying posts

In [45]:
%%file data/flask-examples/app6.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Flask, jsonify, request, abort


app = Flask(__name__)

state = {
    'posts': { }
#         uuid4().hex: {
#             'postedDate': datetime.utcnow(),
#             'authorName': 'rick',
#             'title': 'first!',
#             'body': 'First post!'
#         }
#     }
}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('get_posts')})

@app.route('/post')
def get_posts():
    post_links = [url_for('get_post', id=id) for id in state['posts']]
    return jsonify(
        _links={'self': url_for('get_posts')},
        data=[dict(_links=dict(self=link)) for link in post_links])

@app.route('/post', methods=['POST'])
def create_post():
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        'request_args': request.args,
        **request.json
    }
    # if you don't like the **syntax, you can also post.update(request.json)
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('get_post', id=post_id)
    return result, 201

@app.route('/post/<id>')
def get_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    return jsonify_post(id, post)

@app.route('/post/<id>', methods=['PUT'])
def update_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json  # python 3.5?6?
    }
    # post.update(request.json)
    state['posts'][id] = post
    return jsonify_post(id, post)

@app.route('/post/<id>', methods=['DELETE'])
def delete_post(id):
    state['posts'].pop(id, None)
    return '', 204

def jsonify_post(id, post, **kwargs):
    return jsonify(
        _links={'self': url_for('get_post', id=id)},
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body'],
        request_args=post.get('request_args', None)
    )
    

Overwriting data/flask-examples/app6.py


In [47]:
from pprint import pprint

sess = requests.Session()
sess.auth = ('rick', 'password')

with running_app('data.flask-examples.app6'):
    root = sess.get('http://localhost:5000').json()
    posts_url = glom(root, '_links.posts')
    resp = sess.post(
        posts_url, 
        json=dict(title='First!', body="First post!"),
        params={'something else': 'entirely'}
    )
    pprint(resp.headers['Location'])
    input('Press enter to continue')
    post1 = resp.json()
    pprint(post1)
    input('Press enter to continue')
    sess.post(posts_url, json=dict(title='Second!', body="Second post!"))
    sess.post(posts_url, json=dict(title='Third!', body="Another post!"))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')
    resp = sess.delete(glom(post1, '_links.self'))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')


 * Serving Flask app "data.flask-examples.app6"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:28:21] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:28:21] "POST /post?something+else=entirely HTTP/1.1" 201 -
'http://localhost:5000/post/4dc16dd17fc94e42bbec6a21fad8cf86'
Press enter to continue
{'_links': {'self': 'http://localhost:5000/post/4dc16dd17fc94e42bbec6a21fad8cf86'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2020-07-29T17:28:21.042144',
 'request_args': {'something else': 'entirely'},
 'title': 'First!'}
Press enter to continue
127.0.0.1 - - [29/Jul/2020 10:28:56] "POST /post HTTP/1.1" 201 -
127.0.0.1 - - [29/Jul/2020 10:28:56] "POST /post HTTP/1.1" 201 -
127.0.0.1 - - [29/Jul/2020 10:28:56] "GET /post HTTP/1.1" 200 -
{'_links': {'self': 'http://localhost:5000/post'},
 'data': [{'_links': {'self': 'http://localhost:5000/post/4dc16dd17fc94e42bbec6a21fad8cf86'}},
  

# Organizing code into blueprints for reusability and maintenance

In [56]:
%%file data/flask-examples/app7.py
from flask import Flask, jsonify, url_for

from . import app7_posts
from .app7_util import url_for

app = Flask(__name__)

app.register_blueprint(app7_posts.mod, url_prefix='/post')

@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('posts.get_root')})

Overwriting data/flask-examples/app7.py


In [57]:
%%file data/flask-examples/app7_util.py
import flask

state = {'posts': {}}

def url_for(*args, **kwargs):
    return flask.url_for(*args, _external=True, **kwargs)

Overwriting data/flask-examples/app7_util.py


In [58]:
%%file data/flask-examples/app7_posts.py
from datetime import datetime
from uuid import uuid4

import flask
from flask import Blueprint, jsonify, request, abort

from .app7_util import state, url_for

mod = Blueprint('posts', __name__)

@mod.route('')
def get_root():
    post_links = [url_for('.get_post', id=id) for id in state['posts']]
    return jsonify(
        _links={
            'self': url_for('.get_root'),
            'home': url_for('get_root'),
        },
        data=[dict(_links=dict(self=link)) for link in post_links])

@mod.route('', methods=['POST'])
def create_post():
    post_id = uuid4().hex
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    }
    state['posts'][post_id] = post
    result = jsonify_post(post_id, post)
    result.headers['Location'] = url_for('.get_post', id=post_id)
    return result

@mod.route('<id>')
def get_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    return jsonify_post(id, post)

@mod.route('<id>', methods=['PUT'])
def update_post(id):
    post = state['posts'].get(id)
    if not post:
        abort(404)
    post = {
        'postedDate': datetime.utcnow(),
        'authorName': request.authorization.username,
        **request.json
    }
    state['posts'][id] = post
    return jsonify_post(id, post)

@mod.route('<id>', methods=['DELETE'])
def delete_post(id):
    state['posts'].pop(id)
    return '', 204

def jsonify_post(id, post, **kwargs):
    return jsonify(
        _links={'self': url_for('.get_post', id=id)},
        postedDate=post['postedDate'].isoformat(),
        authorName=post['authorName'],
        title=post['title'],
        body=post['body']
    )

Overwriting data/flask-examples/app7_posts.py


In [59]:
from pprint import pprint

sess = requests.Session()
sess.auth = ('rick', 'password')
sess.headers['Content-Type'] = 'application/json'

with running_app('data.flask-examples.app7'):
    root = sess.get('http://localhost:5000').json()
    posts_url = glom(root, '_links.posts')
    resp = sess.post(posts_url, json=dict(title='First!', body="First post!"))
    pprint(resp.headers['Location'])
    input('Press enter to continue')
    post1 = resp.json()
    pprint(post1)
    input('Press enter to continue')
    sess.post(posts_url, json=dict(title='Second!', body="Second post!"))
    sess.post(posts_url, json=dict(title='Third!', body="Another post!"))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')
    resp = sess.delete(glom(post1, '_links.self'))
    pprint(sess.get(posts_url).json())
    input('Press enter to continue')


 * Serving Flask app "data.flask-examples.app7"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2020 10:35:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:35:40] "POST /post HTTP/1.1" 200 -
'http://localhost:5000/post/ef3aa45433a24a8c919d20145deec191'
Press enter to continue
{'_links': {'self': 'http://localhost:5000/post/ef3aa45433a24a8c919d20145deec191'},
 'authorName': 'rick',
 'body': 'First post!',
 'postedDate': '2020-07-29T17:35:40.498585',
 'title': 'First!'}
Press enter to continue
127.0.0.1 - - [29/Jul/2020 10:36:00] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:36:00] "POST /post HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2020 10:36:00] "GET /post HTTP/1.1" 200 -
{'_links': {'home': 'http://localhost:5000/',
            'self': 'http://localhost:5000/post'},
 'data': [{'_links': {'self': 'http://localhost:5000/post/ef3aa45433a24a8c919d20145deec191'}},
          {'_links': {'self':

In [60]:
ls data/flask-examples/

__pycache__/               lab_blog.py
app1.py                    mongo-app-withcomments.py
app2.py                    mongo-app.py
app3.py                    mongo_model.py
app4.py                    sa-app-withcomments.py
app5.py                    sa-app.py
app6.py                    sa_model.py
app7.py                    state.pkl
app7_posts.py              state.py
app7_util.py               templates/
blog.db                    todo-soln.py
bp_authors.py              todo.py
bp_comments.py             ui1.py
bp_posts.py                ui2.py
flask_helpers.py           util.py


# Lab

Open the [Flask APIs lab][flask-api-lab]

[flask-api-lab]: ./flask-api-lab.ipynb